# 🔍 RAG 기본 구조 이해하기 - WebLoader 활용

## 📚 개요

**RAG(Retrieval-Augmented Generation)** 는 외부 문서에서 관련 정보를 **검색(Retrieve)** 하고, 그 정보를 바탕으로 **AI가 답변을 생성(Generate)** 하는 혁신적인 기술입니다! 📖✨

이 튜토리얼에서는 **네이버 뉴스 기사**를 활용한 실제 RAG 시스템을 구축해보며, RAG의 핵심 개념과 구현 방법을 단계별로 학습합니다.

## 📋 목차

1. **🛠️ 환경 설정**
2. **📖 RAG 기본 구조 이해**
   - 사전 작업 단계 (1~4단계)
   - RAG 실행 단계 (5~8단계)
3. **🏗️ 네이버 뉴스 QA 챗봇 구축**
   - 웹 페이지 로딩
   - 텍스트 분할
   - 벡터 임베딩
   - 검색기 설정
   - 프롬프트 정의
   - LLM 체인 구성
4. **🚀 실제 질문-답변 테스트**

---

## 1. 사전작업(Pre-processing) - 1~4 단계

![rag-1.png](./assets/rag-1.png)

![rag-1-graphic](./assets/rag-graphic-1.png)

사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행합니다.

- **1단계 문서로드(Document Load)**: 문서 내용을 불러옵니다.
- **2단계 분할(Text Split)**: 문서를 특정 기준(Chunk) 으로 분할합니다.
- **3단계 임베딩(Embedding)**: 분할된(Chunk) 를 임베딩하여 저장합니다.
- **4단계 벡터DB 저장**: 임베딩된 Chunk 를 DB에 저장합니다.

## 2. RAG 수행(RunTime) - 5~8 단계

![rag-2.png](./assets/rag-2.png)

![](./assets/rag-graphic-2.png)

- **5단계 검색기(Retriever)**: 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의합니다. 리트리버는 검색 알고리즘이며(Dense, Sparse) 리트리버로 나뉘게 됩니다. 
  - **Dense**: 유사도 기반 검색(FAISS, DPR)
  - **Sparse**: 키워드 기반 검색(BM25, TF-IDF)
- **6단계 프롬프트**: RAG 를 수행하기 위한 프롬프트를 생성합니다. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됩니다. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있습니다.
- **7단계 LLM**: 모델을 정의합니다.(GPT-3.5, GPT-4, Claude, etc..)
- **8단계 Chain**: 프롬프트 - LLM - 출력 에 이르는 체인을 생성합니다.

## 🛠️ 환경 설정

RAG 시스템을 구축하기 전에 필요한 **API 키**와 **추적 도구**를 설정해봅시다! 🔧

### 🔑 API 키 설정

**OpenAI API 키**를 안전하게 관리하기 위해 `.env` 파일을 사용합니다. 이 방법으로 중요한 정보가 코드에 직접 노출되는 것을 방지할 수 있습니다! 🛡️

> **💡 참고**: `.env` 파일에 `OPENAI_API_KEY=your_api_key_here` 형식으로 저장하세요.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

### 📊 LangSmith 추적 설정

**LangSmith**는 LangChain 애플리케이션의 실행 과정을 **실시간으로 추적하고 분석**할 수 있는 강력한 도구입니다! 🔍

#### 🤔 왜 LangSmith를 사용할까요?

LangChain으로 구축한 RAG 시스템은 여러 단계를 거쳐 동작합니다:
- **문서 검색** → **프롬프트 생성** → **LLM 호출** → **결과 반환**

이 과정에서 **어느 단계에서 문제가 발생했는지**, **각 단계의 성능은 어떤지** 파악하기 어려울 수 있습니다.

LangSmith는 이 모든 과정을 **시각화**하고 **추적**하여 **디버깅과 최적화**를 훨씬 쉽게 만들어줍니다! 📈

> **💡 참고**: LangSmith는 선택사항이지만, 학습 과정에서 각 단계를 명확하게 볼 수 있어 매우 유용합니다.

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## 🏗️ 네이버 뉴스 기반 QA 챗봇 구축

이번 튜토리얼에서는 **네이버 뉴스 기사**의 내용에 대해 자연스럽게 질문할 수 있는 **뉴스 기사 QA 챗봇**을 구축해보겠습니다! 📰🤖

### 🎯 무엇을 만들까요?

- **📖 뉴스 기사 분석**: 실제 네이버 뉴스 기사를 자동으로 읽고 이해
- **❓ 자연어 질문**: "부영그룹의 출산 장려 정책은 뭐야?" 같은 자연스러운 질문
- **💡 정확한 답변**: 기사 내용을 바탕으로 한 정확하고 맥락적인 답변

### 🛠️ 사용할 도구들

- **🤖 OpenAI ChatGPT**: 답변 생성을 위한 대화형 AI 모델
- **🔍 OpenAI Embeddings**: 텍스트를 벡터로 변환하는 임베딩 모델  
- **📚 FAISS**: 빠른 유사도 검색을 위한 벡터 저장소

### 🚀 구현 과정 미리보기

이 가이드에서는 **약 20줄의 코드**로 간단하면서도 강력한 인덱싱 파이프라인과 RAG 체인을 완성할 수 있습니다!

### 📦 필요한 라이브러리 가져오기

RAG 시스템 구축에 필요한 핵심 라이브러리들을 임포트합니다. 각 라이브러리의 역할을 알아보세요! 🧰

#### 🔧 주요 라이브러리 설명

- **`bs4` (BeautifulSoup4)**: 웹 페이지의 HTML을 파싱하여 원하는 텍스트만 깔끔하게 추출 🕷️
- **`langchain`**: AI 애플리케이션을 위한 핵심 기능들
  - `RecursiveCharacterTextSplitter`: 긴 텍스트를 적절한 크기로 나누는 **똑똑한 가위** ✂️
  - `WebBaseLoader`: 웹 페이지 내용을 자동으로 읽어오는 **웹 크롤러** 🌐
  - `FAISS`: 수백만 개의 벡터도 빠르게 검색하는 **초고속 검색엔진** ⚡
  - `StrOutputParser`: AI 모델의 복잡한 출력을 간단한 텍스트로 변환 📝
  - `RunnablePassthrough`: 데이터를 다음 단계로 전달하는 **중계소** 🚛
- **`langchain_openai`**: OpenAI의 강력한 AI 기능들
  - `ChatOpenAI`: 대화형 AI 모델 (GPT-4 등) 🤖
  - `OpenAIEmbeddings`: 텍스트를 벡터로 변환하는 임베딩 모델 🔢

#### 💡 왜 이런 라이브러리들이 필요할까요?

웹 페이지에서 정보를 읽고 → 적절히 나누고 → 벡터로 변환하고 → 검색하고 → AI가 답변하는 전체 과정을 자동화하기 위해서입니다!

In [ ]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

### 🌐 1단계: 웹 페이지 내용 로딩

웹 페이지에서 텍스트를 추출하고, 이를 **적절한 크기의 청크**로 나누어 **인덱싱**하는 과정입니다.

#### 🎯 WebBaseLoader의 똑똑한 기능

`WebBaseLoader`는 웹 페이지 전체를 가져오는 대신, **필요한 부분만 정확히 추출**할 수 있습니다!

**`bs4.SoupStrainer`** 를 사용하면:
- 🎯 **선택적 추출**: 광고, 메뉴, 댓글 등은 제외하고 **본문만** 가져오기
- ⚡ **빠른 처리**: 불필요한 HTML 파싱 시간 단축
- 🧹 **깔끔한 데이터**: 노이즈 없는 깨끗한 텍스트

#### 🔍 SoupStrainer 사용 예시

```python
bs4.SoupStrainer(
    "div",  # HTML 태그 지정
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},  # CSS 클래스 지정
)
```

이렇게 하면 **네이버 뉴스의 제목과 본문**만 정확히 추출할 수 있습니다! 📰✨

In [ ]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
# 웹 페이지 로더 설정: 특정 URL에서 HTML 콘텐츠를 가져옴
loader = WebBaseLoader(
    web_paths=(
        "https://n.news.naver.com/article/437/0000378416",
    ),  # 로드할 웹 페이지 URL
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",  # div 태그만 파싱
            attrs={
                "class": ["newsct_article _article_body", "media_end_head_title"]
            },  # 뉴스 본문과 제목 클래스만 추출
        )
    ),
)

# 웹 페이지에서 문서 로드 실행
docs = loader.load()
print(f"문서의 수: {len(docs)}")  # 로드된 문서 개수 출력
docs  # 문서 내용 확인

### ✂️ 2단계: 텍스트 분할 (Text Splitting)

**`RecursiveCharacterTextSplitter`** 는 긴 텍스트를 **AI가 처리하기 좋은 크기**로 나누는 똑똑한 도구입니다!

#### 🤔 왜 텍스트를 나눠야 할까요?

- **🧠 AI 모델의 제한**: 한 번에 처리할 수 있는 텍스트 길이에 한계가 있음
- **🎯 정확한 검색**: 작은 단위로 나누면 관련 정보를 더 정확히 찾을 수 있음  
- **💰 비용 효율성**: 필요한 부분만 AI에게 전달하여 비용 절약

#### ⚙️ 주요 파라미터

- **`chunk_size=1000`**: 각 청크의 **최대 크기** (글자 수)
- **`chunk_overlap=100`**: 청크 간 **겹치는 부분** (맥락 보존용)

In [ ]:
# 텍스트 분할기 설정: 긴 문서를 작은 청크로 나누기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 크기 (글자 수)
    chunk_overlap=100,  # 청크 간 겹치는 부분 (맥락 보존)
)

# 로드된 문서들을 청크로 분할
splits = text_splitter.split_documents(docs)
len(splits)  # 분할된 청크의 개수 출력

### 🔢 3단계: 벡터 임베딩 및 저장

**벡터 저장소(Vector Store)**는 텍스트를 **숫자로 변환하여 저장**하는 똑똑한 데이터베이스입니다!

#### 🎯 FAISS vs Chroma 비교

| 특징 | **FAISS** ⚡ | **Chroma** 🎨 |
|------|-------------|-------------|
| **속도** | 초고속 (Facebook 개발) | 빠름 |
| **메모리** | 인메모리 (휘발성) | 영구 저장 가능 |
| **사용 용도** | 실험, 프로토타입 | 프로덕션 |

#### 🔍 임베딩이 뭔가요?

**임베딩**은 텍스트를 **의미를 담은 숫자 배열**로 변환하는 것입니다:

```
"부영그룹 출산 장려" → [0.1, -0.3, 0.8, 0.2, ...]
```

비슷한 의미를 가진 텍스트들은 **비슷한 숫자 패턴**을 가지게 되어, 컴퓨터가 **의미적 유사도**를 계산할 수 있게 됩니다! 🧮✨

In [ ]:
# 벡터스토어를 생성합니다.
# FAISS 벡터 저장소 생성: 문서 청크들을 벡터로 변환하여 저장
vectorstore = FAISS.from_documents(
    documents=splits,  # 분할된 문서 청크들
    embedding=OpenAIEmbeddings(
        model="text-embedding-3-small"
    ),  # OpenAI 임베딩 모델 사용
)

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
# 벡터 저장소에서 유사도 기반 검색을 수행하는 검색기 생성
retriever = vectorstore.as_retriever()

### 🔗 4단계: 검색기(Retriever)와 프롬프트 설정

**검색기(Retriever)**는 사용자의 질문과 **가장 관련 있는 문서 조각들**을 찾아주는 똑똑한 검색엔진입니다! 🕵️‍♂️

#### 🎯 검색기의 동작 원리

1. **질문 벡터화**: 사용자 질문을 숫자 배열로 변환
2. **유사도 계산**: 저장된 문서들과 얼마나 비슷한지 계산
3. **상위 결과 반환**: 가장 관련 있는 문서 조각들을 순서대로 반환

#### 🤖 프롬프트의 역할

**프롬프트**는 AI에게 **"이렇게 답변해줘"** 라고 지시하는 설명서입니다:

- **📋 역할 정의**: "당신은 질문-답변을 수행하는 AI입니다"
- **📖 정보 제공**: 검색된 문서 내용을 `{context}`에 삽입
- **❓ 질문 전달**: 사용자 질문을 `{question}`에 삽입
- **📝 답변 형식**: 어떤 스타일로 답변할지 지정

In [ ]:
# 질문-답변을 위한 프롬프트 템플릿 정의
from langchain_core.prompts import PromptTemplate

# RAG용 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

### 💡 프롬프트 허브 활용하기

**LangChain Hub**에서 미리 만들어진 검증된 프롬프트를 다운로드할 수 있습니다! 🌐

```python
prompt = hub.pull("teddynote/rag-prompt-korean")
```

#### 🚀 허브 사용의 장점

- **⏰ 시간 절약**: 프롬프트 작성 시간 단축
- **✅ 검증됨**: 이미 테스트를 거친 고품질 프롬프트  
- **🌍 공유**: 커뮤니티에서 만든 다양한 프롬프트 활용
- **🔄 업데이트**: 버전 관리를 통한 지속적인 개선

> **💡 팁**: 직접 만든 프롬프트와 허브 프롬프트를 비교해보며 더 나은 성능을 찾아보세요!

In [ ]:
# prompt = hub.pull("teddynote/rag-prompt")
# prompt

In [ ]:
# ChatOpenAI 모델 초기화
llm = ChatOpenAI(
    model_name="gpt-4.1", temperature=0
)  # 일관된 답변을 위해 temperature=0 설정


# RAG 체인을 생성합니다.
# 검색기 -> 프롬프트 -> LLM -> 출력파서 순서로 연결된 체인 구성
rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }  # 검색된 문서와 질문을 프롬프트에 전달
    | prompt  # 프롬프트 템플릿 적용
    | llm  # LLM 모델로 답변 생성
    | StrOutputParser()  # 출력 결과를 문자열로 파싱
)

### 🚀 스트리밍 출력으로 실시간 응답

**스트리밍 출력**을 사용하면 AI가 답변을 생성하는 과정을 **실시간으로** 볼 수 있습니다! ⚡

#### 🎬 스트리밍 vs 일반 출력

- **일반 출력**: 답변이 완전히 생성된 후 한 번에 표시 😴
- **스트리밍 출력**: 답변을 생성하면서 **단어 하나씩 실시간** 표시 🔥

#### 💡 왜 스트리밍을 사용할까요?

- **⚡ 빠른 반응**: 사용자가 즉시 응답을 확인 가능
- **🎯 사용자 경험**: ChatGPT처럼 자연스러운 대화 느낌
- **🔍 디버깅**: AI의 사고 과정을 단계별로 관찰

`stream_response()` 함수는 스트리밍 출력을 깔끔하게 표시해주는 유틸리티입니다! 🛠️

In [ ]:
# 스트리밍 응답을 위한 유틸리티 함수 임포트
from langchain_teddynote.messages import stream_response

> [LangSmith Trace 보기](https://smith.langchain.com/public/c6047a61-8f44-48e5-89eb-b1e8a6321cea/r)


In [ ]:
# RAG 체인 실행: 부영그룹의 출산 장려 정책에 대한 질문
answer = rag_chain.stream("부영그룹의 출산 장려 정책에 대해 설명해주세요.")
stream_response(answer)  # 스트리밍 방식으로 답변 출력

> [LangSmith Trace 보기](https://smith.langchain.com/public/ed21d80e-b4da-4a08-823b-ed980db9c347/r)


In [ ]:
# RAG 체인 실행: 부영그룹의 출산 지원 금액에 대한 구체적인 질문
answer = rag_chain.stream("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?")

> [LangSmith Trace 보기](https://smith.langchain.com/public/df80c528-61d6-4c83-986a-3373a4039dae/r)


In [ ]:
# RAG 체인 실행: 정부의 저출생 대책을 불릿 포인트 형식으로 요청
answer = rag_chain.stream("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요.")
stream_response(answer)  # 스트리밍 방식으로 답변 출력

> [LangSmith Trace 보기](https://smith.langchain.com/public/1a613ee7-6eaa-482f-a45f-8c22b4e60fbf/r)


In [ ]:
# RAG 체인 실행: 부영그룹의 임직원 수에 대한 질문
answer = rag_chain.stream("부영그룹의 임직원 숫자는 몇명인가요?")
stream_response(answer)  # 스트리밍 방식으로 답변 출력